In [137]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:
from Ising import Ising
import torch
import time

In [139]:
system_size = torch.tensor([16])

In [140]:
iham = Ising(system_size=system_size, periodic=True)

/home/spandan/Projects/tqs/Hamiltonian_utils.py:44: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [141]:
start = time.time()
# data_path = "TFIM_ground_states\\2024-07-24T16-19-00.994"
data_path = "TFIM_ground_states/2024-07-24T19-26-39.836"
iham.load_dataset(data_path)
end = time.time()
print(end - start)
iham.dataset

Loaded dataset for system size 16 from TFIM_ground_states/2024-07-24T19-26-39.836/16.arrow.
(h_min, h_step, h_max) = (0.5, 0.01, 1.5).
0.0009157657623291016


,N,h,energy,state
0,16,0.50,-17.016712,"[0.6178205144880193, 0.078518229753998, 0.0785..."
1,16,0.51,-17.058544,"[0.6141643161264388, 0.07967155361508126, 0.07..."
2,16,0.52,-17.101269,"[0.6104305568436547, 0.08079908755634686, 0.08..."
3,16,0.53,-17.144893,"[0.6050290837460097, 0.08168557953788275, 0.08..."
4,16,0.54,-17.189421,"[0.6047395928869441, 0.08325112357322331, 0.08..."
...,...,...,...,...
96,16,1.46,-26.192125,"[0.06880083488412342, 0.030018265353548786, 0...."
97,16,1.47,-26.331392,"[0.06733118217296416, 0.029575886193991142, 0...."
98,16,1.48,-26.470978,"[0.0659154662190349, 0.029146934386637936, 0.0..."
99,16,1.49,-26.610874,"[0.06455105796577158, 0.02873083757832767, 0.0..."


In [142]:
start = time.time()
energy, psi = iham.retrieve_ground(param=[1.5])
print(energy)
print(psi)
end = time.time()
print(end - start, "s")

-26.751073763765437
tensor([0.0632, 0.0283, 0.0283,  ..., 0.0283, 0.0283, 0.0632],
       dtype=torch.float64)
0.0012640953063964844 s


In [143]:
spins = iham.basis
spins

tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 1,  ..., 0, 1, 1],
        [0, 1, 0,  ..., 1, 0, 1]], device='cuda:0')

## Batching Across $\vec J$ s for one $\lambda \vec J.\hat H(\vec J)$


Might be a good idea to produce a PyTorch dataset object that is associated with Hamiltonians.

- Acts as a wrapper for memoized ground states for a Hamiltonian
- Should be able to be randomized (and randomization should apply across batches)
- Ground states should be able to be retrieved in a batched manner
- In this sense, a Hamiltonian _is_ a dataset


In [144]:
iham.load_dataset(data_path)

Loaded dataset for system size 16 from TFIM_ground_states/2024-07-24T19-26-39.836/16.arrow.
(h_min, h_step, h_max) = (0.5, 0.01, 1.5).


In [145]:
dataframe = iham.dataset
dataframe

,N,h,energy,state
0,16,0.50,-17.016712,"[0.6178205144880193, 0.078518229753998, 0.0785..."
1,16,0.51,-17.058544,"[0.6141643161264388, 0.07967155361508126, 0.07..."
2,16,0.52,-17.101269,"[0.6104305568436547, 0.08079908755634686, 0.08..."
3,16,0.53,-17.144893,"[0.6050290837460097, 0.08168557953788275, 0.08..."
4,16,0.54,-17.189421,"[0.6047395928869441, 0.08325112357322331, 0.08..."
...,...,...,...,...
96,16,1.46,-26.192125,"[0.06880083488412342, 0.030018265353548786, 0...."
97,16,1.47,-26.331392,"[0.06733118217296416, 0.029575886193991142, 0...."
98,16,1.48,-26.470978,"[0.0659154662190349, 0.029146934386637936, 0.0..."
99,16,1.49,-26.610874,"[0.06455105796577158, 0.02873083757832767, 0.0..."


In [146]:
ground_state_tensor = torch.tensor(dataframe["state"].tolist())
ground_state_tensor

tensor([[0.6178, 0.0785, 0.0785,  ..., 0.0785, 0.0785, 0.6178],
        [0.6142, 0.0797, 0.0797,  ..., 0.0797, 0.0797, 0.6142],
        [0.6104, 0.0808, 0.0808,  ..., 0.0808, 0.0808, 0.6104],
        ...,
        [0.0659, 0.0291, 0.0291,  ..., 0.0291, 0.0291, 0.0659],
        [0.0646, 0.0287, 0.0287,  ..., 0.0287, 0.0287, 0.0646],
        [0.0632, 0.0283, 0.0283,  ..., 0.0283, 0.0283, 0.0632]],
       dtype=torch.float64)

In [147]:
# keep it row-major BUT two-dimensional in case we have multiple params
param_tensor = torch.tensor(dataframe[["h"]].to_numpy().transpose())
param_tensor

tensor([[0.5000, 0.5100, 0.5200, 0.5300, 0.5400, 0.5500, 0.5600, 0.5700, 0.5800,
         0.5900, 0.6000, 0.6100, 0.6200, 0.6300, 0.6400, 0.6500, 0.6600, 0.6700,
         0.6800, 0.6900, 0.7000, 0.7100, 0.7200, 0.7300, 0.7400, 0.7500, 0.7600,
         0.7700, 0.7800, 0.7900, 0.8000, 0.8100, 0.8200, 0.8300, 0.8400, 0.8500,
         0.8600, 0.8700, 0.8800, 0.8900, 0.9000, 0.9100, 0.9200, 0.9300, 0.9400,
         0.9500, 0.9600, 0.9700, 0.9800, 0.9900, 1.0000, 1.0100, 1.0200, 1.0300,
         1.0400, 1.0500, 1.0600, 1.0700, 1.0800, 1.0900, 1.1000, 1.1100, 1.1200,
         1.1300, 1.1400, 1.1500, 1.1600, 1.1700, 1.1800, 1.1900, 1.2000, 1.2100,
         1.2200, 1.2300, 1.2400, 1.2500, 1.2600, 1.2700, 1.2800, 1.2900, 1.3000,
         1.3100, 1.3200, 1.3300, 1.3400, 1.3500, 1.3600, 1.3700, 1.3800, 1.3900,
         1.4000, 1.4100, 1.4200, 1.4300, 1.4400, 1.4500, 1.4600, 1.4700, 1.4800,
         1.4900, 1.5000]], dtype=torch.float64)

In [148]:
# Parameters are indexed by rows, selecting all entries along the column dimension:
param_tensor[:, 0]

tensor([0.5000], dtype=torch.float64)

In [149]:
basis = iham.basis

In [150]:
iham.param_range

tensor([[0.5000],
        [1.5000]])

In [151]:
iham.param_dim

1

## Rewriting init_seq for various parameters


In [152]:
phys_dim = 2
n_params = 2
n_dim = 1
input_dim = phys_dim + n_dim + 1 + 1 + n_params
seq_prefix_len = n_dim + n_params
system_size = torch.tensor([[16]])
n = system_size.prod(dim=1)
param_offset = phys_dim + n_dim + 2

batch_size = 7

In [193]:
import plotly.graph_objects as go
import numpy as np


def plot_tensor(tens, labels, opacity=0.7):

    x = np.arange(tens.shape[0])
    y = np.arange(tens.shape[1])
    z = np.arange(tens.shape[2])

    xlen = len(x)
    ylen = len(y)
    zlen = len(z)

    print(f"(x, y, z) = ({xlen}, {ylen}, {zlen})")

    X, Y, Z = np.meshgrid(x, y, z)

    color_function = np.vectorize(lambda x, y, z: tens[x, y, z])

    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=X.flatten(),
                y=Y.flatten(),
                z=Z.flatten(),
                mode="markers",
                marker=dict(
                    size=5,
                    # color=tens.swapaxes(1, 2)
                    # .swapaxes(0, 2)
                    # .swapaxes(1, 2)
                    # .flatten(),  # set color to an array/list of desired values
                    color=color_function(X, Y, Z).flatten(),
                    colorscale="Viridis",  # choose a colorscale
                    opacity=opacity,
                ),
            )
        ]
    )

    fig.update_layout(
        scene=dict(xaxis_title=labels[0], yaxis_title=labels[1], zaxis_title=labels[2]),
    )
    fig.show()

In [154]:
testflat = torch.arange(8).reshape(2, 2, 2)
testflat

tensor([[[0, 1],
         [2, 3]],

        [[4, 5],
         [6, 7]]])

In [155]:
testflat[0]

tensor([[0, 1],
        [2, 3]])

In [156]:
testflat.flatten(0)

tensor([0, 1, 2, 3, 4, 5, 6, 7])

In [157]:
test_prefix_encoding = torch.zeros(seq_prefix_len, batch_size, input_dim)

In [158]:
plot_tensor(test_prefix_encoding, ["Sequence Prefix", "Batch", "Input Dim"])

(x, y, z) = (3, 7, 7)


In [159]:
# order of coloring: input dim, then sequence prefix, then batch
# swap 0 and 2, then 1 and 2

In [160]:
test_params = [
    (torch.tensor([0, 1, 2]), torch.tensor([1.5, 2.4])),
    (torch.tensor([3, 4]), torch.tensor([1.0, 0.3])),
]

test_params

[(tensor([0, 1, 2]), tensor([1.5000, 2.4000])),
 (tensor([3, 4]), tensor([1.0000, 0.3000]))]

In [161]:
# TODO: does .clone mean that the tensor is copied in its entirety? Documentation just
# states that the function is differentiable and does not specify whether tensors
# placed by view are copied or not.

In [162]:
indices, values = test_params[0]
values_diag = torch.diag(values)
values_diag

tensor([[1.5000, 0.0000],
        [0.0000, 2.4000]])

In [163]:
indices

tensor([0, 1, 2])

In [164]:
prefix_dim_start = n_dim
prefix_dim_end = prefix_dim_start + n_params
input_dim_start = phys_dim + n_dim + 2  # This is param_offset
input_dim_end = input_dim_start + n_params

test_prefix_encoding[
    prefix_dim_start:prefix_dim_end, indices, input_dim_start:input_dim_end
] = values_diag.unsqueeze(1)

In [165]:
test_highlight = test_prefix_encoding.clone()

In [166]:
test_highlight[
    prefix_dim_start:prefix_dim_end, indices, input_dim_start:input_dim_end
] += (torch.diag(torch.tensor(100).repeat(2)).unsqueeze(1).to(torch.float32))

In [167]:
torch.diag(torch.tensor(100).repeat(2))

tensor([[100,   0],
        [  0, 100]])

In [168]:
test_highlight[prefix_dim_start:prefix_dim_end, indices, input_dim_start:input_dim_end]

tensor([[[101.5000,   0.0000],
         [101.5000,   0.0000],
         [101.5000,   0.0000]],

        [[  0.0000, 102.4000],
         [  0.0000, 102.4000],
         [  0.0000, 102.4000]]])

In [169]:
# This plotting function doesn't seem to be accurate
plot_tensor(test_highlight, ["Prefix Dim", "Batch", "Input Dim"])

(x, y, z) = (3, 7, 7)


In [170]:
test_highlight

tensor([[[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000]],

        [[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000, 101.5000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000, 101.5000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000, 101.5000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.00

In [171]:
test_highlight[:, 0, :]

tensor([[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000, 101.5000,   0.0000],
        [  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000, 102.4000]])

These are the right dimensions!


In [172]:
test_prefix_encoding

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.5000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.5000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.5000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 

In [173]:
plot_tensor(test_prefix_encoding, ["Sequence Prefix", "Batch", "Input Dim"])

(x, y, z) = (3, 7, 7)


In [174]:
values_diag.unsqueeze(1)

tensor([[[1.5000, 0.0000]],

        [[0.0000, 2.4000]]])

In [175]:
def write_params_to_prefix(indices_values, prefix_encoding):
    for indices, values in indices_values:
        values_diag = torch.diag(values)
        prefix_dim_start = n_dim
        prefix_dim_end = prefix_dim_start + n_params
        input_dim_start = phys_dim + n_dim + 2  # This is param_offset
        input_dim_end = input_dim_start + n_params
        prefix_encoding[
            prefix_dim_start:prefix_dim_end, indices, input_dim_start:input_dim_end
        ] = values_diag.unsqueeze(1)

    return prefix_encoding

In [176]:
test = torch.zeros(seq_prefix_len, batch_size, input_dim)
test

tensor([[[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.]]])

In [177]:
indices, values = test_params[0], test_params[1]
res = write_params_to_prefix(test_params, test)
res

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.5000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.5000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.5000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 

In [178]:
res[1][0][5]

tensor(1.5000)

In [179]:
def wrap_spins_batch(indices_params, prefix_len, spins, phys_dim, system_size):
    n, batch_size = spins.shape
    seq_encoding = torch.zeros(prefix_len + n, batch_size, input_dim)

    size_input = torch.diag(system_size.log())
    parity = system_size % 2

    seq_encoding[:n_dim, :, phys_dim : (phys_dim + n_dim)] = size_input.unsqueeze(1)

    seq_encoding[:n_dim, :, phys_dim + n_dim] = parity.unsqueeze(1)

    seq_encoding = write_params_to_prefix(indices_params, seq_encoding)

    seq_encoding[prefix_len:, :, :phys_dim] = torch.functional.F.one_hot(
        spins.to(torch.int64), num_classes=phys_dim
    )

    return seq_encoding

In [180]:
batch_size

7

In [181]:
iham.basis

tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 1,  ..., 0, 1, 1],
        [0, 1, 0,  ..., 1, 0, 1]], device='cuda:0')

In [182]:
test_spins = iham.basis[:, :batch_size]
test_spins

tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 1, 1, 0, 0, 1],
        [0, 1, 0, 1, 0, 1, 0]], device='cuda:0')

In [183]:
test_spins.to(torch.int64)

tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 1, 1, 0, 0, 1],
        [0, 1, 0, 1, 0, 1, 0]], device='cuda:0')

In [184]:
test_encoded_spins = torch.functional.F.one_hot(
    test_spins.to(torch.int64), num_classes=phys_dim
)
print(test_encoded_spins)
print(test_encoded_spins.shape)

tensor([[[1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0]],

        [[1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0]],

        [[1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0]],

        [[1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0]],

        [[1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0]],

        [[1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0]],

        [[1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0]],

        [[1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0],
         [1, 0]],

        [[1, 0],
         [1, 0]

In [185]:
test_params

[(tensor([0, 1, 2]), tensor([1.5000, 2.4000])),
 (tensor([3, 4]), tensor([1.0000, 0.3000]))]

In [186]:
seq_prefix_len

3

In [187]:
test_spins

tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 1, 1, 0, 0, 1],
        [0, 1, 0, 1, 0, 1, 0]], device='cuda:0')

In [203]:
mask_test_params = [
    (torch.tensor([0, 1, 2]), torch.tensor([100, 100]).to(torch.float32)),
    (torch.tensor([3, 4]), torch.tensor([100, 100]).to(torch.float32)),
]

In [230]:
start = time.time()
final_test_spins = wrap_spins_batch(
    test_params, seq_prefix_len, test_spins, phys_dim, system_size
)
end = time.time()
print(end - start)

0.0004909038543701172


In [207]:
test_params

[(tensor([0, 1, 2]), tensor([1.5000, 2.4000])),
 (tensor([3, 4]), tensor([1.0000, 0.3000]))]

In [208]:
plot_tensor(final_test_spins, ["Sequence Prefix", "Batch", "Input Dim"], opacity=0.5)

(x, y, z) = (19, 7, 7)


In [209]:
from model import TransformerModel

In [210]:
system_size

tensor([[16]])

In [223]:
# Sizes of Ising chains to explore
system_sizes = np.arange(10, 17, 2).reshape(-1, 1)

# Ising Hamiltonians
Hamiltonians = [Ising(system_size_i, periodic=False) for system_size_i in system_sizes]

# dim(J)
param_dim = 2  # Hamiltonians[0].param_dim
# Number of embedding dimensions (e.g., after passing through grey layers)
embedding_size = 32
# Transformer hyperparameters
n_head = 8  # Attention heads
n_hid = embedding_size  # Number of hidden units in the feedforward network
n_layers = 8  # Number of transformer layers
dropout = 0  # Dropout rate
minibatch = 10000  # Batch size

model = TransformerModel(
    system_sizes,
    param_dim,
    embedding_size,
    n_head,
    n_hid,
    n_layers,
    dropout=dropout,
    minibatch=minibatch,
)

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning:

enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer



In [224]:
test_spins

tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 1, 1, 0, 0, 1],
        [0, 1, 0, 1, 0, 1, 0]], device='cuda:0')

In [225]:
test_params

[(tensor([0, 1, 2]), tensor([1.5000, 2.4000])),
 (tensor([3, 4]), tensor([1.0000, 0.3000]))]

In [226]:
model.set_param(param=test_params[0][1], system_size=system_size[0])
wrapped_spins = model.wrap_spins(test_spins)

In [227]:
plot_tensor(wrapped_spins, ["Sequence Prefix", "Batch", "Input Dim"], opacity=0.5)

(x, y, z) = (19, 7, 7)


In [228]:
plot_tensor(final_test_spins, ["Sequence Prefix", "Batch", "Input Dim"], opacity=0.5)

(x, y, z) = (19, 7, 7)


# Old Code


In [ ]:
def generate_index_sequential(dataset_len, n, batch_size):
    # Iterate over flattened ground states (this abstract sequence contains all basis
    # elements for all points in parameter space)
    total_sites = dataset_len * n
    for i in range(0, dataset_len, batch_size):
        # The flattened index that corresponds to this batch
        flattened_idx = torch.arange(i, min(i + batch_size, total_sites))

        # The index of the ground state in the dataset (i.e., the index of the
        # sampling point in parameter space)
        ground_state_idx = torch.div(flattened_idx, n, rounding_mode="floor")

        # The index of the site in the ground state
        site_idx = torch.fmod(flattened_idx, n)
        yield ground_state_idx, site_idx

In [ ]:
gen = generate_index_sequential(100, 16, 10)

In [ ]:
print(next(gen))

(tensor([2, 2, 2, 2, 2, 2, 2, 2, 3, 3]), tensor([ 8,  9, 10, 11, 12, 13, 14, 15,  0,  1]))


In [ ]:
# Now use the tensor indices to index into the dataset
df = iham.dataset
df

,N,h,energy,state
0,16,0.50,-17.016712,"[0.6178205144880193, 0.078518229753998, 0.0785..."
1,16,0.51,-17.058544,"[0.6141643161264388, 0.07967155361508126, 0.07..."
2,16,0.52,-17.101269,"[0.6104305568436547, 0.08079908755634686, 0.08..."
3,16,0.53,-17.144893,"[0.6050290837460097, 0.08168557953788275, 0.08..."
4,16,0.54,-17.189421,"[0.6047395928869441, 0.08325112357322331, 0.08..."
...,...,...,...,...
96,16,1.46,-26.192125,"[0.06880083488412342, 0.030018265353548786, 0...."
97,16,1.47,-26.331392,"[0.06733118217296416, 0.029575886193991142, 0...."
98,16,1.48,-26.470978,"[0.0659154662190349, 0.029146934386637936, 0.0..."
99,16,1.49,-26.610874,"[0.06455105796577158, 0.02873083757832767, 0.0..."


In [ ]:
from torch.utils.data import DataLoader, Dataset, IterableDataset

In [ ]:
# Generates a basis tensor of dimension (n, 2^n) where each column is a basis state
# denoting the spin of each site in the system (1 = up, 0 = down)
def generate_basis_binary_representation(n):
    basis = torch.zeros(n, 2**n)
    for i in range(2**n):
        basis[:, i] = torch.tensor([int(x) for x in list(bin(i)[2:].zfill(n))])
    return basis

In [ ]:
generate_basis_binary_representation(20)

tensor([[0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 1.,  ..., 0., 1., 1.],
        [0., 1., 0.,  ..., 1., 0., 1.]])

In [ ]:
class IsingIterableDatasetSequential(IterableDataset):
    def __init__(self, dataframe, batch_size, basis):
        """
        Parameters:
            dataframe: pd.DataFrame
                The dataset containing the ground states. Should allow for .iloc indexing.
            batch_size: int
                The number of sites to sample in each batch
            basis: torch.Tensor (n, 2**n)
                All energy eigenstates of the Hamiltonian (i.e., all possible spin sequences).
        """

        self.dataframe = dataframe
        self.ground_state_tensor = torch.tensor(
            dataframe["state"].tolist(), device="cpu"
        )

        self.param_tensor = torch.tensor(dataframe["h"], device="cpu")

        # Number of sites in the system
        self.n = basis.shape[0]

        self.batch_size = batch_size
        self.basis = basis

        # Ground state tensor dimensions. The second (ground_state_length)
        # is important for flattened indexing.
        self.dataset_len = self.ground_state_tensor.shape[0]
        self.ground_state_length = 2**self.n

        # The number of labels (probability amplitudes) in the dataset
        self.total_prob_amps = self.dataset_len * self.ground_state_length

    def __iter__(self):
        # Iterate over probability amplitude entries in batch_size steps
        for i in range(0, self.dataset_len * self.ground_state_length, self.batch_size):

            # Flattened index of the start and end of the batch
            start_idx = i
            end_idx = min(i + self.batch_size, self.total_prob_amps)
            labels = self.ground_state_tensor.view(-1)[start_idx:end_idx]

            # The row index of the start and end of the batch (i.e., indices of
            # the ground states from which the labels and parameters come)
            start_row_idx = start_idx // self.ground_state_length
            end_row_idx = (end_idx // self.ground_state_length) + 1  # [ ) convention
            params = self.param_tensor[start_row_idx:end_row_idx]

            # The column indices from which the batch is drawn (i.e., indices of the
            # basis states from which the inputs come)
            batch_idx = torch.arange(start_idx, end_idx)
            batch_idx = torch.remainder(batch_idx, self.ground_state_length)
            basis_states = self.basis[:, batch_idx]

            param_indices = torch.arange(

            # num_first_parameter = self.ground_state_length - (
            #     start_idx % self.ground_state_length
            # )

            # num_of_parameters_in_middle = (
            #     self.batch_size - num_first_parameter
            # ) // self.ground_state_length

            # num_last_parameter = (
            #     self.batch_size
            #     - num_first_parameter
            #     - num_of_parameters_in_middle * self.ground_state_length
            # )

            # param_indices = torch.arange(0, self.batch_size).split(
            #     [num_first_parameter, num_of_parameters_in_middle, num_last_parameter]
            #     if num_of_parameters_in_middle > 0
            #     else [num_first_parameter, num_last_parameter]
            # )

            # param_indices = list(param_indices)

            yield basis_states, params, param_indices, labels

In [ ]:
class IsingIterableDatasetOrdered(IterableDataset):
    def __init__(self, dataframe, batch_size, basis):

In [ ]:
isingds = IsingIterableDatasetSequential(iham.dataset, 16, iham.basis)

In [ ]:
torch.arange(0, 10).split([3, 4, 3])[1]

tensor([3, 4, 5, 6])

In [ ]:
for batch in isingds:
    print(batch)
    break

RuntimeError: split_with_sizes expects split_sizes to sum exactly to 16 (input tensor's size at dimension 0), but got split_sizes=[65536, 16]

In [ ]:
test = torch.arange(0, 4)
test

tensor([0, 1, 2, 3])

In [ ]:
test[[-1]]

tensor([3])

In [ ]:
torch.stack((torch.arange(0, 10), torch.arange(0, 10)))

tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [ ]:
test[1:-1]

tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [ ]:
batch_size = 100
n = iham.basis.shape[0]
hilbert_dimension = 2**n
dataset_len = iham.dataset.shape[0]
total_prob_amps = dataset_len * hilbert_dimension

tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tens

In [ ]:
torch.arange(0, 100000000, 1)

tensor([       0,        1,        2,  ..., 99999997, 99999998, 99999999])

In [ ]:
def gpu_setup():
    # Setup for PyTorch:
    if torch.cuda.is_available():
        torch_device = torch.device("cuda")
        print("PyTorch is using GPU {}".format(torch.cuda.current_device()))
    else:
        torch_device = torch.device("cpu")
        print("GPU unavailable; using CPU")

In [ ]:
torch.device("cpu")

device(type='cpu')

In [ ]:
iham.n

tensor(16)

In [ ]:
gpu_setup()

PyTorch is using GPU 0


In [ ]:
torch.arange(0, 10, 3, device="cpu")

tensor([0, 3, 6, 9])

In [ ]:
ising_ds = IsingIterableDatasetSequential(df, 10, iham.basis)

In [ ]:
for i in ising_ds:
    print(i)
    break

(tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
        [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]], device='cuda:0'), tensor([0.6178, 0.0785, 0.0785, 0.0200, 0.0785, 0.0101, 0.0200, 0.0063, 0.0785,
        0.0100], dtype=torch.float64))


In [ ]:
test1 = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
test2 = 4
res = torch.remainder(test1, test2)
res

tensor([1, 2, 3, 0, 1, 2, 3, 0, 1, 2])

In [ ]:
example_spin

In [ ]:
torch.functional.F.one_hot()